# Mask R-CNN - Evaluation on Prostate Dataset


This notebook shows how to use trained Mask R-CNN on prostate dataset for evaluation. As for large pathology image, we crop each image to several patches. This notebook is designed to get the detection reulst for single patch and evaluate them use mIOU. You'd need a GPU, though, because the network backbone is a Resnet101, which would be slow to detect on a CPU.

The code of the Prostate dataset can be found in prostate.py.

## Import Module

In [1]:
# import module from system lib
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scipy.io

In [2]:
# import module from maskrcnn repo
from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate

%matplotlib inline

# Specify GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


## Configurations

In [3]:
# Specify the dir that store the prostate dataset
dataset_dir = os.path.join(os.path.dirname(os.getcwd()), "Data_Pre_Processing/cedars-224")
# We do 5-fold validation, specify which fold to be exclude for the current run
held_out_set = 4
# Featch the mean_pixel based on the training data (data exclude the held_out_set)
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
# Configuration
class EvaluationConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5
    DETECTION_NMS_THRESHOLD = 1     
    MEAN_PIXEL = np.array(mean_pixel)
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_DIM = 512
    DETECTION_CROP = [128, 384, 128, 384] # [height_crop_start, height_crop_end, width_crop_start, width_crop_end]
    MODE = 16
evaluation_config = EvaluationConfig()
evaluation_config.display()


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_CROP                 [128, 384, 128, 384]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        1
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 193.97800579  120.89113632  183.79060979]
MINI_MASK_SHAPE                (56, 56)
MODE                           16
NAME                           prostate
NUM_CLASSES       

## Create Model Graph and Loading Weights

In [4]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="detection", 
                          config=evaluation_config,
                          model_dir=MODEL_DIR)
# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights
h5_filename = None # Specify the h5 filename here if you want to choose a specific file
epoch = -1

if h5_filename is not None:
    model_path = os.path.join(ROOT_DIR, ".h5 file name here")
elif epoch == -1:    
    model_path = model.find_last()[1]
else:
    try:
        model_path = model.find_specific(epoch = epoch, held_out_set = held_out_set)[1]
    except:
        model_path = model.find_specific(epoch = epoch)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /scratch/wenyuan/Mask_RCNN_On_Pathology/Mask_RCNN/logs/prostate20180223T1554-held-out-4/mask_rcnn_prostate_0071.h5


## Prepare the Dataset

In [5]:
dataset_val = prostate.ProstateDataset()
_, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)
# val_list = [image for image in val_list if image not in exclude_list]
dataset_val.load_prostate(dataset_dir, val_list, mode = 16)
dataset_val.prepare()

## Run Evaluation

In [ ]:
# Initialize the confusion matrix
C_MATRIX = np.zeros((4, 4))
# Threshold for evaluating the mIOU
Threshold = 0.5
# Create crop region
hv, wv = utils.create_crop_region(evaluation_config) # meshgrid for crop region
# Process display setting
display_step = 10 # print out process for every display_step images
total_image = len(val_list)
rc_num = int(math.sqrt(evaluation_config.MODE)) # how many patches in each row or col
for image_id in range(0, len(dataset_val.image_ids), 16):
    gt_mask_whole = []
    det_mask_whole = []
    for i in range(evaluation_config.MODE):
        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
                modellib.load_image_gt(dataset_val, evaluation_config,
                                       image_id + i, use_mini_mask=False)
        # Convert gt-instance mask to gt-sementic mask
        gt_sementic_mask = utils.instance_2_sementic(gt_mask, gt_class_id)
        gt_sementic_mask = gt_sementic_mask['ATmask'][hv, wv] # crop the label
        gt_mask_whole.append(gt_sementic_mask)
        # Run object detection
        results = model.detect([image], verbose=0)
        # TODO: create a function for evaluation in model
        r = results[0]    
        if np.argmax(r['tumor_probs']) == 1:
            det_sementic_mask = r['sementic_mask']
        else:
            det_sementic_mask = np.zeros((image.shape[0], image.shape[1]))
        det_sementic_mask = det_sementic_mask[hv, wv] # crop the detection
        det_mask_whole.append(det_sementic_mask)
    ann = utils.combine_2_whole_slide(gt_mask_whole, rc_num, rc_num)
    det = utils.combine_2_whole_slide(det_mask_whole, rc_num, rc_num)
    # Compute confusion matrix
    c_matrix = confusion_matrix(np.ravel(ann), np.ravel(det))
    # Expand the c_matrix to NUM_CLASSES * NUM_CLASSES
    c_matrix = utils.expand_c_matrix(c_matrix, evaluation_config.NUM_CLASSES, ann, det)    
    # Compute whether the IOU is below the threshold
    mIOU, IOU, below_th = utils.compute_mIOU(c_matrix, th = Threshold)
    if below_th:
        print("Number of image is:", val_list[int(image_id / 16)])
        print("mIOU: ", mIOU, "IOU:", IOU)
    # Display the process
    if ((image_id + 1) % (evaluation_config.MODE * display_step) == 0):
        print('Done evaluating %d / %d!\n'%((image_id + 1) / evaluation_config.MODE, total_image))

Number of image is: 185
mIOU:  0.309812564818 IOU: [0.3567275054254076, 0, 0.88236172973449944, 0.00016102411336097581]
Number of image is: 7
mIOU:  0.207182060755 IOU: [0.12302101643555098, 0, 0.70570722658499863, 0]
Number of image is: 406
mIOU:  0.390507991118 IOU: [0.37746318724853622, 0.15613358169238956, 0.55401198955241104, 0.47442320597968202]
Number of image is: 196
mIOU:  0.226583410432 IOU: [0.20493004776673535, 0.0, 0.70140359396273266, 0]
